In [ ]:

import numpy as np # linear algebra
import re, os
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split


In [ ]:

datapath = '/content/drive/MyDrive/ML_files/BBC/bbc/' 
for dirname, _, filenames in os.walk(datapath):
  for filename in filenames:
    print(os.path.join(dirname, filename))    

/content/drive/MyDrive/ML_files/BBC/bbc/README.TXT
/content/drive/MyDrive/ML_files/BBC/bbc/business/018.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/001.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/003.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/004.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/002.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/009.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/008.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/015.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/017.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/011.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/013.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/010.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/012.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/014.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/016.txt
/content/drive/MyDrive/ML_files/BBC/bbc/business/007.txt
/content/drive/MyDrive/ML_files/BBC/b

In [ ]:
# Step 1 - Get the file details
directory = []
file = []
title = []
text = []
label = []

for dirname, _ , filenames in os.walk(datapath):
    print('Directory: ', dirname)
    print('Subdir: ', dirname.split('/')[-1])
    # remove the Readme.txt file
    # will not find file in the second iteration so we skip the error
    try:
        filenames.remove('README.TXT')
    except:
        pass
    for filename in filenames:
        directory.append(dirname)
        file.append(filename)
        label.append(dirname.split('/')[-1])
        #print(filename)
        fullpathfile = os.path.join(dirname,filename)
        with open(fullpathfile, 'r', encoding="utf8", errors='ignore') as infile:
            intext = ''
            firstline = True
            for line in infile:
                if firstline:
                    title.append(line.replace('\n',''))
                    firstline = False
                else:
                    intext = intext + ' ' + line.replace('\n','')
            text.append(intext)

#

Directory:  /content/drive/MyDrive/ML_files/BBC/bbc/
Subdir:  
Directory:  /content/drive/MyDrive/ML_files/BBC/bbc/business
Subdir:  business
Directory:  /content/drive/MyDrive/ML_files/BBC/bbc/entertainment
Subdir:  entertainment
Directory:  /content/drive/MyDrive/ML_files/BBC/bbc/politics
Subdir:  politics
Directory:  /content/drive/MyDrive/ML_files/BBC/bbc/sport
Subdir:  sport
Directory:  /content/drive/MyDrive/ML_files/BBC/bbc/tech
Subdir:  tech


In [ ]:
fulldf = pd.DataFrame(list(zip(directory, file, title, text, label)), 
               columns =['directory', 'file', 'title', 'text', 'label'])

df = fulldf.filter(['title','text','label'], axis=1)

print("FullDf : ", fulldf.shape)
print("DF : ", df.shape)

FullDf :  (2225, 5)
DF :  (2225, 3)


In [ ]:
df.head()

,title,text,label
0,India's rupee hits five-year high,India's rupee has hit a five-year high after...,business
1,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarn...,business
2,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yu...,business
3,High fuel prices hit BA's profits,British Airways has blamed high fuel prices ...,business
4,Dollar gains on Greenspan speech,The dollar has hit its highest level against...,business


In [ ]:
print("Title: ",df['title'][0]," Label: ",df['label'][0])
print("Text:\n")
df['text'][0]

Title:  India's rupee hits five-year high  Label:  business
Text:



'  India\'s rupee has hit a five-year high after Standard & Poor\'s (S&P) raised the country\'s foreign currency rating.  The rupee climbed to 43.305 per US dollar on Thursday, up from a close of 43.41. The currency has gained almost 1% in the past three sessions. S&P, which rates borrowers\' creditworthiness, lifted India\'s rating by one notch to \'BB+\'. With Indian assets now seen as less of a gamble, more cash is expected to flow into its markets, buoying the rupee.  "The upgrade is positive and basically people will use it as an excuse to come back to India," said Bhanu Baweja, a strategist at UBS. "Money has moved out from India in the first two or three weeks of January into other markets like Korea and Thailand and this upgrade should lead to a reversal." India\'s foreign currency rating is now one notch below investment grade, which starts at \'BBB-\'. The increase has put it on the same level as Romania, Egypt and El Salvador, and one level below Russia.'

In [ ]:
print("Text:\n",df['text'][0])

Text:
   India's rupee has hit a five-year high after Standard & Poor's (S&P) raised the country's foreign currency rating.  The rupee climbed to 43.305 per US dollar on Thursday, up from a close of 43.41. The currency has gained almost 1% in the past three sessions. S&P, which rates borrowers' creditworthiness, lifted India's rating by one notch to 'BB+'. With Indian assets now seen as less of a gamble, more cash is expected to flow into its markets, buoying the rupee.  "The upgrade is positive and basically people will use it as an excuse to come back to India," said Bhanu Baweja, a strategist at UBS. "Money has moved out from India in the first two or three weeks of January into other markets like Korea and Thailand and this upgrade should lead to a reversal." India's foreign currency rating is now one notch below investment grade, which starts at 'BBB-'. The increase has put it on the same level as Romania, Egypt and El Salvador, and one level below Russia.


In [ ]:
X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

print(X_train.shape)
print(X_test.shape)

(1557,)
(668,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_transformer = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r'[A-Za-z]+',
                                   ngram_range=(1, 2), lowercase=True, max_features=150000)

In [ ]:
X_train = text_transformer.fit_transform(X_train)
X_test = text_transformer.transform(X_test)

print("X Train DF: ",X_train.shape)
print("X Test DF: ", X_test.shape)

X Train DF:  (1557, 150000)
X Test DF:  (668, 150000)


In [ ]:
X_train

<1557x150000 sparse matrix of type '<class 'numpy.float64'>'
	with 414989 stored elements in Compressed Sparse Row format>

In [ ]:
text_transformer.get_feature_names()

['aaa',
 'aaa battery',
 'aaa indoor',
 'aaa m',
 'aaas',
 'aaas american',
 'aaas champion',
 'aaas indoor',
 'aaas long',
 'aaas title',
 'aac',
 'aac baddeley',
 'aac ed',
 'aac k',
 'aadc',
 'aadc oversee',
 'aadc said',
 'aadc set',
 'aaltra',
 'aaltra belgian',
 'aamir',
 'aamir khan',
 'aan',
 'aan teachings',
 'aara',
 'aara association',
 'aarhus',
 'aarhus stiftstidende',
 'aaron',
 'aaron mauger',
 'aaron mokoena',
 'aaron persico',
 'abacus',
 'abacus magazine',
 'abandon',
 'abandon draft',
 'abandon fixed',
 'abandon pursuit',
 'abandoned',
 'abandoned appeal',
 'abandoned calls',
 'abandoned career',
 'abandoned dog',
 'abandoned flying',
 'abandoned good',
 'abandoned suggesting',
 'abandoned wake',
 'abandoning',
 'abandoning calls',
 'abandoning carefully',
 'abandoning duty',
 'abandoning labour',
 'abandoning plans',
 'abandonment',
 'abandonment abortion',
 'abandonment option',
 'abandonment peso',
 'abate',
 'abate markos',
 'abating',
 'abating annual',
 'abba',

In [ ]:
!pip install eli5

     |████████████████████████████████| 106 kB 28.6 MB/s 


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
import eli5
from IPython.display import Image

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial',
                           random_state=17, n_jobs=4)

In [ ]:
logit.fit(X_train,y_train)

LogisticRegression(C=50.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=4, penalty='l2',
                   random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
eli5.show_weights(estimator=logit, 
                  feature_names= text_transformer.get_feature_names(),top=(50, 5))

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report 

sv = LinearSVC()
sv.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
pred = sv.predict(X_test)

In [ ]:
print(classification_report(y_test,pred))

               precision    recall  f1-score   support

     business       0.99      0.94      0.96       159
entertainment       0.99      0.98      0.99       120
     politics       0.95      1.00      0.97       115
        sport       0.99      0.99      0.99       146
         tech       0.98      0.98      0.98       128

     accuracy                           0.98       668
    macro avg       0.98      0.98      0.98       668
 weighted avg       0.98      0.98      0.98       668



In [ ]:
type(X_test[0])

scipy.sparse.csr.csr_matrix

In [ ]:
 path = '/content/temp2.txt'
 file = open(path,'r',encoding="utf-8",errors='ignore')
 t1 = []
 intext = ''
  
 for line in file:
   intext = intext + ' ' + line.replace('\n','')
 
 t1.append(intext)

In [ ]:
t1 

[' Isinbayeva claims new world best  Pole vaulter Yelena Isinbayeva broke her own indoor world record by clearing 4.89 metres in Lievin on Saturday.  It was the Russian\'s 12th world record of her career and came just a few days after she cleared 4.88m at the Norwich Union Grand Prix in Birmingham. The Olympic champion went on to attempt 5.05m at the meeting on France but failed to clear that height. In the men\'s 60m, former Olympic 100m champion Maurice Greene could only finish second to Leonard Scott. It was Greene\'s second consecutive defeat at the hands of his fellow American, who also won in Birmingham last week. "I ran my race perfectly," said Scott, who won in 6.46secs, his best time indoors. "I am happy even if I know that Maurice is a long way from being at his peak at the start of the season."']

In [ ]:
t1 = text_transformer.transform(t1)
#pred = logit.predict(t1)
#print(pred)

In [ ]:
print(sv.predict(t1))

['sport']


In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
def clean_text(path):
  file = open(path,'r',encoding="utf-8",errors='ignore')
  t1 = []
  intext = ''

  for line in file:
    intext = intext + ' ' + line.replace('\n','')
  # removing punctuation,numbers and extra special chars
  words = re.sub("[^A-Za-z]"," ",intext)
  
  # converting words to lowercase and splitting to individual words
  words = words.lower().split()

  # searching set of stopwords
  stops = set(stopwords.words("english"))

  # Removing stopwords
  imp_words = [w for w in words if not w in stops]

  # Joining words back into one string seperated by space
  file.close()
  return (" ".join(imp_words))

In [ ]:
 cleantx=[]
 path = '/content/temp2.txt'
 #file = open(path,'r',encoding="utf-8",errors='ignore')
 strin=clean_text(path)
 cleantx.append(strin)
 t = text_transformer.transform(cleantx)
 pred = sv.predict(t)
 print(pred)

['sport']


In [ ]:
print(strin)

recent bharatiya janata party national executive meeting prime minister narendra modi took jibe congress stating party run family culture public welfare paradoxically days earlier bjp finalised alliance assembly polls uttar pradesh seven smaller parties previously part front named hissedari morcha pact routine strategic move build poll especially principal bjp rival samajwadi party firmed alliance suheldev bharatiya samaj party sbsp brought wake smaller outfits however ironical parties lined bjp besides caste appeal among different numerically lesser social groups also nascent individual centric parties little semblance democratic functioning led individuals either branched small parties recently set shop
